In [19]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
from model import RFModel
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

InteractiveShell.ast_node_interactivity = "all"

In [10]:
with open("/home/jason/bot_model/my_clean_data_training.csv") as f:
    training_df = pd.read_csv(f, sep=",")

with open("/home/jason/bot_model/my_clean_data_normies.csv") as f:
    normal_df = pd.read_csv(f, sep=",")
normal_df["target"] = "normal"

df = training_df.append(normal_df)
training_df.shape
normal_df.shape
df.shape
df.columns

(19686, 25)

(181617, 25)

(201303, 25)

Index(['no_follow', 'link_id', 'author', 'author_verified',
       'author_comment_karma', 'author_link_karma', 'created_utc', 'over_18',
       'body', 'is_submitter', 'recent_num_comments',
       'recent_num_last_30_days', 'recent_avg_no_follow', 'recent_avg_gilded',
       'recent_avg_responses', 'recent_percent_neg_score', 'recent_avg_score',
       'recent_min_score', 'recent_avg_controversiality', 'recent_avg_ups',
       'recent_avg_diff_ratio', 'recent_max_diff_ratio',
       'recent_avg_sentiment_polarity', 'recent_min_sentiment_polarity',
       'target'],
      dtype='object')

In [11]:
# drop duplicates
df = df.drop_duplicates(subset=["author", "link_id", "created_utc"])

# correct labeling
df.loc[df.author == "PoliticsModeratorBot", "target"] = "bot"

# Label known bots in normies
bot_authors = df[df.target == "bot"].author.unique()
df.loc[((df.target == "normal") & (df.author.isin(bot_authors))), "target"] = "bot"

In [12]:
# Delete irrelevant columns
columns = ['link_id', 'author', 'created_utc', 'body']
df.drop(columns, inplace=True, axis=1)
df.shape
df.columns

(181617, 21)

Index(['no_follow', 'author_verified', 'author_comment_karma',
       'author_link_karma', 'over_18', 'is_submitter', 'recent_num_comments',
       'recent_num_last_30_days', 'recent_avg_no_follow', 'recent_avg_gilded',
       'recent_avg_responses', 'recent_percent_neg_score', 'recent_avg_score',
       'recent_min_score', 'recent_avg_controversiality', 'recent_avg_ups',
       'recent_avg_diff_ratio', 'recent_max_diff_ratio',
       'recent_avg_sentiment_polarity', 'recent_min_sentiment_polarity',
       'target'],
      dtype='object')

In [13]:
df[df["target"] == "normal"].describe()
df[df["target"] == "bot"].describe()
df[df["target"] == "troll"].describe()

,author_comment_karma,author_link_karma,recent_num_comments,recent_num_last_30_days,recent_avg_no_follow,recent_avg_gilded,recent_avg_responses,recent_percent_neg_score,recent_avg_score,recent_min_score,recent_avg_controversiality,recent_avg_ups,recent_avg_diff_ratio,recent_max_diff_ratio,recent_avg_sentiment_polarity,recent_min_sentiment_polarity
count,512.000000,512.000000,508.000000,508.000000,508.000000,508.0,508.000000,508.000000,508.000000,508.000000,508.000000,508.000000,508.000000,508.000000,508.000000,508.000000
mean,221.740234,287.814453,10.625984,8.498031,0.801948,0.0,134.868440,8.001282,3.332161,-6.602362,0.014552,3.332161,0.579638,0.702373,0.069456,-0.092790
std,415.189181,4296.991318,7.462323,7.322791,0.290430,0.0,452.258113,19.878838,8.825358,22.613365,0.040222,8.825358,0.365658,0.364490,0.206760,0.264495
min,-100.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-36.000000,-131.000000,0.000000,-36.000000,0.000000,0.000000,-0.500000,-0.750000
25%,2.000000,1.000000,3.000000,2.000000,0.746711,0.0,4.000000,0.000000,1.000000,-1.000000,0.000000,1.000000,0.190042,0.333333,0.000000,-0.200000
50%,32.000000,1.000000,10.000000,6.000000,0.900000,0.0,14.272727,0.000000,2.100000,1.000000,0.000000,2.100000,0.643162,0.965259,0.000000,0.000000
75%,199.000000,1.000000,20.000000,15.000000,1.000000,0.0,66.625000,5.263158,3.425000,1.000000,0.000000,3.425000,0.986458,1.000000,0.093750,0.000000
max,1490.000000,68830.000000,20.000000,20.000000,1.000000,0.0,3781.333333,100.000000,55.500000,43.000000,0.333333,55.500000,1.000000,1.000000,0.837500,0.825000


,author_comment_karma,author_link_karma,recent_num_comments,recent_num_last_30_days,recent_avg_no_follow,recent_avg_gilded,recent_avg_responses,recent_percent_neg_score,recent_avg_score,recent_min_score,recent_avg_controversiality,recent_avg_ups,recent_avg_diff_ratio,recent_max_diff_ratio,recent_avg_sentiment_polarity,recent_min_sentiment_polarity
count,174554.000000,174554.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000,174548.000000
mean,33221.934880,12954.338079,19.645186,19.179062,0.900532,0.000122,346.092293,3.140029,3.846306,-1.062120,0.014111,3.846306,0.598152,0.776085,0.048254,-0.159397
std,86162.573382,76764.558540,2.180702,3.235332,0.164953,0.002831,1312.982855,8.847076,11.806418,8.882108,0.042829,11.806418,0.290287,0.253076,0.201430,0.313849
min,-100.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-49.500000,-360.000000,0.000000,-49.500000,0.000000,0.000000,-0.788046,-1.000000
25%,572.000000,1.000000,20.000000,20.000000,0.850000,0.000000,6.900000,0.000000,1.000000,0.000000,0.000000,1.000000,0.363750,0.613333,0.000000,-0.375000
50%,3136.000000,5.000000,20.000000,20.000000,1.000000,0.000000,48.000000,0.000000,1.300000,1.000000,0.000000,1.300000,0.621832,0.891304,0.017953,-0.055556
75%,18933.000000,93.000000,20.000000,20.000000,1.000000,0.000000,246.862500,0.000000,2.650000,1.000000,0.000000,2.650000,0.872224,0.975000,0.130553,0.000000
max,582974.000000,849774.000000,20.000000,20.000000,1.000000,0.333333,22185.550000,100.000000,618.000000,348.000000,1.000000,618.000000,1.000000,1.000000,1.000000,1.000000


,author_comment_karma,author_link_karma,recent_num_comments,recent_num_last_30_days,recent_avg_no_follow,recent_avg_gilded,recent_avg_responses,recent_percent_neg_score,recent_avg_score,recent_min_score,recent_avg_controversiality,recent_avg_ups,recent_avg_diff_ratio,recent_max_diff_ratio,recent_avg_sentiment_polarity,recent_min_sentiment_polarity
count,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.0,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000,6551.000000
mean,1875.858342,13269.686307,17.216150,16.214318,0.908674,0.0,480.305800,8.157243,17.637774,-3.707220,0.037963,17.637774,0.178670,0.338531,0.114331,-0.417152
std,5485.144438,17001.106087,5.833464,6.349888,0.177314,0.0,844.265580,12.169791,232.859592,10.614497,0.070244,232.859592,0.063770,0.126035,0.097439,0.287326
min,-20.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-10.000000,-145.000000,0.000000,-10.000000,0.000000,0.000000,-0.444792,-1.000000
25%,108.000000,366.000000,20.000000,14.000000,0.900000,0.0,33.807353,0.000000,1.000000,-5.000000,0.000000,1.000000,0.150000,0.290977,0.045441,-0.625000
50%,453.000000,1487.000000,20.000000,20.000000,0.950000,0.0,138.050000,5.000000,1.500000,-1.000000,0.000000,1.500000,0.184301,0.333333,0.107986,-0.400000
75%,1138.000000,20663.000000,20.000000,20.000000,1.000000,0.0,568.925000,10.000000,2.650000,0.000000,0.050000,2.650000,0.214267,0.378915,0.175428,-0.200000
max,32373.000000,45367.000000,20.000000,20.000000,1.000000,0.0,8548.000000,100.000000,4330.850000,174.000000,1.000000,4330.850000,0.800000,1.000000,0.775000,0.550000


In [23]:
X = df.drop(["target"], axis=1)
y = df["target"].values

X.fillna(X_train.mean(), inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [24]:
model = DecisionTreeClassifier(
    max_depth=3,
    class_weight={"normal": 1, "bot": 2.5, "troll": 5},
    min_samples_leaf=100,
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

feature_importance = pd.Series(
    model.feature_importances_, index=df.columns.drop("target")
).sort_values(ascending=False)
print(feature_importance)

DecisionTreeClassifier(class_weight={'bot': 2.5, 'normal': 1, 'troll': 5},
                       max_depth=3, min_samples_leaf=100)

/home/jason/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jason/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jason/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n         bot       1.00      1.00      1.00     34897\n      normal       0.00      0.00      0.00        99\n       troll       0.97      0.99      0.98      1328\n\n    accuracy                           1.00     36324\n   macro avg       0.66      0.66      0.66     36324\nweighted avg       0.99      1.00      0.99     36324\n'

author_verified                  0.933167
author_comment_karma             0.050583
recent_max_diff_ratio            0.012394
recent_avg_diff_ratio            0.003550
recent_num_comments              0.000307
no_follow                        0.000000
recent_avg_score                 0.000000
recent_avg_sentiment_polarity    0.000000
recent_avg_ups                   0.000000
recent_avg_controversiality      0.000000
recent_min_score                 0.000000
recent_avg_responses             0.000000
recent_percent_neg_score         0.000000
recent_avg_gilded                0.000000
recent_avg_no_follow             0.000000
recent_num_last_30_days          0.000000
is_submitter                     0.000000
over_18                          0.000000
author_link_karma                0.000000
recent_min_sentiment_polarity    0.000000
dtype: float64


In [25]:
print(classification_report(y_test, y_pred))

/home/jason/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jason/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         bot       1.00      1.00      1.00     34897
      normal       0.00      0.00      0.00        99
       troll       0.97      0.99      0.98      1328

    accuracy                           1.00     36324
   macro avg       0.66      0.66      0.66     36324
weighted avg       0.99      1.00      0.99     36324



/home/jason/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
